## Context
Cars4U is a budding tech start-up that aims to find footholes in the Used car market as pre-owned car market has continued to grow over the past years (specifically between 2018-19). Unlike New Cars where pricingh and supply are failrly deterministic and managed by OEMs (Original Equipment Manufacturer / except for dealership level discounts, used cars have huge uncertainty in both pricing and supply. Therefore, it is very important for Cars4U as a start-up to be able to determine what its future supply and pricing will be like in order to manage its future growth and keep a foothole in ever growing Used car market. 

## Objective
Analyze the data and come up with a pricing model that can effectively predict the price of used cars and can help the business in devising profitable strategies using differential pricing. 

## Key Questions
1) What will be the market price in order to plan target price to not sell below market value?

 # **Data PreProcessing**
  ____________________

`@author mirali - Created on Friday Feb 3 - Start your work from the next line`

<h3 style="color:Brown">Loading and Exploring Data</h3>

In [4]:
# this will help in making the Python code more structured automatically (good coding practice)
%load_ext nb_black
# To Suppress warning such as future warnings
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
# For reading and manipulating data
import pandas as pd
import numpy as np

# Libraries used for data visualization
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

# Command to tell Python to actually display the graphs
%matplotlib inline
# Remove the limit from the number of displayed columns and rows so i can see the entire df when printing
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

# Libraries used in Data Split (to test and train)
from sklearn.model_selection import train_test_split

# Libraries used for Linear Regression Model Building
from sklearn.linear_model import LinearRegression

<IPython.core.display.Javascript object>

In [5]:
# Importing data into data Variable which is our original Data
data = pd.read_csv(
    "https://raw.githubusercontent.com/miricoder/projects/main/Supervised_Learning_Regression/used_cars_data.csv"
)
# Creating Copy of The original Data
df = data.copy()

<IPython.core.display.Javascript object>

In [6]:
# Display column and row values using pyton 3's f-string
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Looking at the 10 random sample rows done using np.random
# Because looking at random values will give you better understaning than head or tail values
np.random.seed(1)
df.sample(10)

There are 7253 rows and 14 columns.


,S.No.,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
2397,2397,Ford EcoSport 1.5 Petrol Trend,Kolkata,2016,21460,Petrol,Manual,First,17.0 kmpl,1497 CC,121.36 bhp,5.0,9.47,6.00
3777,3777,Maruti Wagon R VXI 1.2,Kochi,2015,49818,Petrol,Manual,First,21.5 kmpl,1197 CC,81.80 bhp,5.0,5.44,4.11
4425,4425,Ford Endeavour 4x2 XLT,Hyderabad,2007,130000,Diesel,Manual,First,13.1 kmpl,2499 CC,141 bhp,7.0,35.29,6.00
3661,3661,Mercedes-Benz E-Class E250 CDI Avantgrade,Coimbatore,2016,39753,Diesel,Automatic,First,13.0 kmpl,2143 CC,201.1 bhp,5.0,86.97,35.28
4514,4514,Hyundai Xcent 1.2 Kappa AT SX Option,Kochi,2016,45560,Petrol,Automatic,First,16.9 kmpl,1197 CC,82 bhp,5.0,8.23,6.34
599,599,Toyota Innova Crysta 2.8 ZX AT,Coimbatore,2019,40674,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,7.0,28.05,24.82
186,186,Mercedes-Benz E-Class E250 CDI Avantgrade,Bangalore,2014,37382,Diesel,Automatic,First,13.0 kmpl,2143 CC,201.1 bhp,5.0,86.97,32.00
305,305,Audi A6 2011-2015 2.0 TDI Premium Plus,Kochi,2014,61726,Diesel,Automatic,First,17.68 kmpl,1968 CC,174.33 bhp,5.0,67.10,20.77
4582,4582,Hyundai i20 1.2 Magna,Kolkata,2011,36000,Petrol,Manual,First,18.5 kmpl,1197 CC,80 bhp,5.0,10.25,2.50
5434,5434,Honda WR-V Edge Edition i-VTEC S,Kochi,2019,13913,Petrol,Manual,First,17.5 kmpl,1199 CC,88.7 bhp,5.0,9.36,8.20


<IPython.core.display.Javascript object>

- After carefully analyzing, column name: `S.No.` is the only column that needs to be dropped reason being:
  * Pandas already provides column id which matches that of S.No and renders S.No's usage pointless 
- `Mileage,Engine,Power` columns needs to be turned into numerics
- Year columns seems ok so far
- Categorical Values like below needs to be applied one-hot encoding closer to the end of our analysis right before model building 
     `Fuel_Type,Transmission,Owner_Type`

**Dropping the S.No Column**

In [7]:
df.drop(["S.No."], axis=1, inplace=True)
# Validating that S.No. column is removed by checking  df.columns
df.columns

Index(['Name', 'Location', 'Year', 'Kilometers_Driven', 'Fuel_Type',
       'Transmission', 'Owner_Type', 'Mileage', 'Engine', 'Power', 'Seats',
       'New_Price', 'Price'],
      dtype='object')

<IPython.core.display.Javascript object>

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7253 entries, 0 to 7252
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               7253 non-null   object 
 1   Location           7253 non-null   object 
 2   Year               7253 non-null   int64  
 3   Kilometers_Driven  7253 non-null   int64  
 4   Fuel_Type          7253 non-null   object 
 5   Transmission       7253 non-null   object 
 6   Owner_Type         7253 non-null   object 
 7   Mileage            7251 non-null   object 
 8   Engine             7207 non-null   object 
 9   Power              7078 non-null   object 
 10  Seats              7200 non-null   float64
 11  New_Price          7253 non-null   float64
 12  Price              6019 non-null   float64
dtypes: float64(3), int64(2), object(8)
memory usage: 736.8+ KB


<IPython.core.display.Javascript object>

- Name column is the first and S.No. column is now gone, our data is down total of 13 columns
- Mileage, Engine, Power, Seats, New_Price and Price all have missing values in them

<h3 style="color:Brown">Processing Data - Before Performing Statisical Analysis</h3>


**Converting `Mileage` column to numeric**

In [9]:
# Looping through to see how many columns have "km/kg" or "kmpl" endings
mileage_endings = ("km/kg", "kmpl")
mileage_cols = []
for colname in df.columns[
    df.dtypes == "object"
]:  # only need to consider string columns
    if (
        df[colname].str.endswith(mileage_endings).any()
    ):  # using `.str` so I can use an element-wise string method
        mileage_cols.append(colname)
print(mileage_cols)

['Mileage']


<IPython.core.display.Javascript object>

In [10]:
def mileage_to_num(mileage_val):
    """
    This function takes in a string representing in Mileage in km/kg & kmpl
    and converts it to a number. For Example, 26.6 km/kg to 26.6
    or 19.67 kmpl to 19.67
    """
    if isinstance(mileage_val, str):  # checks if `mileage_val` is a string
        return float(mileage_val.replace("km/kg", "").replace("kmpl", ""))
    else:
        return np.nan


for colname in mileage_cols:
    df[colname] = df[colname].apply(mileage_to_num)
df[mileage_cols].head()

,Mileage
0,26.60
1,19.67
2,18.20
3,20.77
4,15.20


<IPython.core.display.Javascript object>

**Converting `Engine` column to numeric**

In [11]:
# Looping through to see how many columns have "bhp" endings
engine_endings = "CC"
engine_cols = []
for colname in df.columns[
    df.dtypes == "object"
]:  # only need to consider string columns
    if (
        df[colname].str.endswith(engine_endings).any()
    ):  # using `.str` so I can use an element-wise string method
        engine_cols.append(colname)
print(engine_cols)

['Engine']


<IPython.core.display.Javascript object>

In [12]:
def engine_to_num(engine_val):
    """
    This function takes in a string representing in Mileage in CC
    and converts it to a number.
    """
    if isinstance(engine_val, str):  # checks if `mileage_val` is a string
        return float(engine_val.replace("CC", ""))
    else:
        return np.nan


for colname in engine_cols:
    df[colname] = df[colname].apply(engine_to_num)
df[engine_cols].head()

,Engine
0,998.0
1,1582.0
2,1199.0
3,1248.0
4,1968.0


<IPython.core.display.Javascript object>

**Converting `Power` column to numeric**

In [13]:
# Looping through to see how many columns have "CC" endings
power_endings = "bhp"
power_cols = []
for colname in df.columns[
    df.dtypes == "object"
]:  # only need to consider string columns
    if (
        df[colname].str.endswith(power_endings).any()
    ):  # using `.str` so I can use an element-wise string method
        power_cols.append(colname)
print(power_cols)

['Power']


<IPython.core.display.Javascript object>

In [14]:
def power_to_num(power_val):
    """
    This function takes in a string representing in Mileage in bhp
    and converts it to a number.
    """
    if isinstance(power_val, str):  # checks if `mileage_val` is a string
        return float(power_val.replace("bhp", ""))
    else:
        return np.nan


for colname in power_cols:
    df[colname] = df[colname].apply(power_to_num)
df[power_cols].head()

,Power
0,58.16
1,126.20
2,88.70
3,88.76
4,140.80


<IPython.core.display.Javascript object>

In [15]:
# Printing the random sample from dataset to make sure above numeric conversions has taken affect
print(
    f"There are {df.shape[0]} rows and {df.shape[1]} columns."
)  # Additionally printing the shape of data
# Looking at the 10 random sample rows done using np.random
# Because looking at random values will give you better understaning than head or tail values
np.random.seed(1)
df.sample(10)

There are 7253 rows and 13 columns.


,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
2397,Ford EcoSport 1.5 Petrol Trend,Kolkata,2016,21460,Petrol,Manual,First,17.00,1497.0,121.36,5.0,9.47,6.00
3777,Maruti Wagon R VXI 1.2,Kochi,2015,49818,Petrol,Manual,First,21.50,1197.0,81.80,5.0,5.44,4.11
4425,Ford Endeavour 4x2 XLT,Hyderabad,2007,130000,Diesel,Manual,First,13.10,2499.0,141.00,7.0,35.29,6.00
3661,Mercedes-Benz E-Class E250 CDI Avantgrade,Coimbatore,2016,39753,Diesel,Automatic,First,13.00,2143.0,201.10,5.0,86.97,35.28
4514,Hyundai Xcent 1.2 Kappa AT SX Option,Kochi,2016,45560,Petrol,Automatic,First,16.90,1197.0,82.00,5.0,8.23,6.34
599,Toyota Innova Crysta 2.8 ZX AT,Coimbatore,2019,40674,Diesel,Automatic,First,11.36,2755.0,171.50,7.0,28.05,24.82
186,Mercedes-Benz E-Class E250 CDI Avantgrade,Bangalore,2014,37382,Diesel,Automatic,First,13.00,2143.0,201.10,5.0,86.97,32.00
305,Audi A6 2011-2015 2.0 TDI Premium Plus,Kochi,2014,61726,Diesel,Automatic,First,17.68,1968.0,174.33,5.0,67.10,20.77
4582,Hyundai i20 1.2 Magna,Kolkata,2011,36000,Petrol,Manual,First,18.50,1197.0,80.00,5.0,10.25,2.50
5434,Honda WR-V Edge Edition i-VTEC S,Kochi,2019,13913,Petrol,Manual,First,17.50,1199.0,88.70,5.0,9.36,8.20


<IPython.core.display.Javascript object>

- Data set is down to 13 columns from 14 and Mileage, Power, Enginer columns are now numeric

**Missing Value Treatment**

In [16]:
# Which columns has the most missing values**
df.isnull().sum().sort_values(ascending=False)

Price                1234
Power                 175
Seats                  53
Engine                 46
Mileage                 2
Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
New_Price               0
dtype: int64

<IPython.core.display.Javascript object>

- `Price` column has the most values in it and since Price and its a problem since this is the column we are trying to predict
- `Power, Seats, Engine` also has missing values needs to be looked at
- `Mileage` has the least missing values, still needs to be looked at and treated 

- First 5 rows contains all the missing values

*Understading the pattern before imputing the values*

In [17]:
# counting the number of missing values per row
df.isnull().sum(axis=1).value_counts()

0    5872
1    1308
3      36
2      27
4      10
dtype: int64

<IPython.core.display.Javascript object>

- 5872 have `0` missing values, 1308 has `1` missing values, 34 has `3` missing values, 27 has `2` missing values & 10 has `4` missing values in them

In [23]:
# Checking the rows with 1 missing values
num_missing = df.isnull().sum(axis=1)
df[num_missing == 1].sample(n=5)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
6031,Toyota Corolla H5,Chennai,2007,90000,Petrol,Manual,Third,13.40,1794.0,125.00,5.0,21.085,NaN
6205,Hyundai Santro Xing GL,Ahmedabad,2007,78000,Petrol,Manual,First,0.00,1086.0,62.00,5.0,4.550,NaN
6986,Mercedes-Benz E-Class E240 V6 AT,Mumbai,2003,66800,Petrol,Automatic,Second,11.20,2497.0,203.00,5.0,86.970,NaN
6795,Audi A6 2011-2015 2.0 TDI Premium Plus,Kolkata,2014,60000,Diesel,Automatic,First,17.68,1968.0,174.33,5.0,67.100,NaN
6120,Tata Nano Lx,Hyderabad,2012,25183,Petrol,Manual,First,26.00,624.0,35.00,4.0,7.630,NaN


<IPython.core.display.Javascript object>

- Row 1319 shows that where `Power` is missing `Price` is not, `10` samples also shows the same
- Further more we don't see any rows that comes before price causes price to have a missing value

In [24]:
# Checking the rows with 3 missing values
num_missing = df.isnull().sum(axis=1)
df[num_missing == 3].sample(n=5)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
2074,Maruti Swift 1.3 LXI,Pune,2011,24255,Petrol,Manual,First,16.1,NaN,NaN,NaN,7.88,3.15
1385,Honda City 1.5 GXI,Pune,2004,115000,Petrol,Manual,Second,0.0,NaN,NaN,NaN,13.58,1.50
4952,Fiat Punto 1.4 Emotion,Kolkata,2010,47000,Petrol,Manual,First,14.6,NaN,NaN,NaN,8.60,1.49
2842,Hyundai Santro GLS II - Euro II,Bangalore,2012,43000,Petrol,Manual,First,0.0,NaN,NaN,NaN,4.55,3.25
5185,Maruti Swift 1.3 LXI,Delhi,2012,52000,Petrol,Manual,First,16.1,NaN,NaN,NaN,7.88,3.65


<IPython.core.display.Javascript object>

- Where `Engine` is missing, so does the `Power` & `Seats` rows

In [34]:
# Checking the rows with 2 missing values
num_missing = df.isnull().sum(axis=1)
df[num_missing == 2].sample(n=5)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
2369,Maruti Estilo LXI,Chennai,2008,56000,Petrol,Manual,Second,19.50,1061.0,NaN,NaN,7.880,1.5
7145,Toyota Etios Liva G,Kolkata,2012,37212,Petrol,Manual,First,18.30,1197.0,NaN,5.0,8.525,NaN
6210,Tata Indica DLS,Chennai,2006,140000,Diesel,Manual,Second,13.50,1405.0,NaN,5.0,7.630,NaN
6896,Toyota Etios Liva GD,Hyderabad,2013,86000,Diesel,Manual,First,23.59,1364.0,NaN,5.0,8.525,NaN
6093,Hyundai Santro Xing XL,Bangalore,2007,47000,Petrol,Manual,Second,0.00,1086.0,NaN,5.0,4.550,NaN


<IPython.core.display.Javascript object>

- Not seeing any row missing causing a miss in `Power` but it does however have an affect on price missing like we saw in 1308 rows missing single values

In [32]:
# Checking the rows with 4 missing values
num_missing = df.isnull().sum(axis=1)
df[num_missing == 4].sample(n=5)

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
6643,BMW 5 Series 520d Sedan,Bangalore,2009,150000,Diesel,Automatic,Second,18.48,NaN,NaN,NaN,67.870,NaN
6633,Mahindra TUV 300 P4,Kolkata,2016,27000,Diesel,Manual,First,0.00,NaN,NaN,NaN,18.865,NaN
6902,Toyota Etios Liva V,Kochi,2012,59311,Petrol,Manual,First,18.30,NaN,NaN,NaN,8.525,NaN
6042,Skoda Laura 1.8 TSI Ambition,Bangalore,2009,72000,Petrol,Manual,Second,17.50,NaN,NaN,NaN,25.830,NaN
6541,Toyota Etios Liva Diesel TRD Sportivo,Bangalore,2012,56600,Diesel,Manual,First,23.59,NaN,NaN,NaN,8.525,NaN


<IPython.core.display.Javascript object>

- Where `Engine` is missing so does `Power`,`Seats` and `Price`

There is a pattern which is analyzed programatically as below, and also all missing rows are numerical

In [33]:
for n in num_missing.value_counts().sort_index().index:
    if n > 0:
        print(f"For the rows with exactly {n} missing values, NAs are found in:")
        n_miss_per_col = df[num_missing == n].isnull().sum()
        print(n_miss_per_col[n_miss_per_col > 0])
        print("\n\n")

For the rows with exactly 1 missing values, NAs are found in:
Mileage       2
Power       103
Seats         2
Price      1201
dtype: int64



For the rows with exactly 2 missing values, NAs are found in:
Power    26
Seats     5
Price    23
dtype: int64



For the rows with exactly 3 missing values, NAs are found in:
Engine    36
Power     36
Seats     36
dtype: int64



For the rows with exactly 4 missing values, NAs are found in:
Engine    10
Power     10
Seats     10
Price     10
dtype: int64





<IPython.core.display.Javascript object>

- As we can columns are either missing together or missing in seperate quantities and orders. Price is definetly affected 
- We will not drop these columns as we need them for our model, specially price. We will impute them with thei mean values 

*Imputing the numerical columns values with their mean values, keeping in mind that we will also need to do an Outlier treatment as means are prone to be affected by outliers*

In [47]:
# Using `fillna` with a numeric volumn
# We need to make sure of the column sequence to stay as per data column sequence, otherwise some rows
# won't be impupted into their mean value
numeric_cols = ["Mileage", "Engine", "Power", "Seats", "Price"]
for i in numeric_cols:
    print(df[i].isnull().sum())
    df[i].fillna(df[i].mean(), inplace=True)  # mean imputation
    df[i].isnull().sum()

0
0
0
0
0


<IPython.core.display.Javascript object>

In [48]:
# Which columns has the most missing values**
df.isnull().sum().sort_values(ascending=False)

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
New_Price            0
Price                0
dtype: int64

<IPython.core.display.Javascript object>

- Now `Engine`, `Power`, `Seats`,`Price` are containing 0 missing values

**Basic EDA-Statistical Summary before going any further to see how our data is looking**

In [49]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,7253.0,2013.365366,3.254421,1996.00,2011.00,2014.00,2016.000000,2019.00
Kilometers_Driven,7253.0,58699.063146,84427.720583,171.00,34000.00,53416.00,73000.000000,6500000.00
Mileage,7253.0,18.141580,4.561567,0.00,15.17,18.16,21.100000,33.54
Engine,7253.0,1616.573470,593.394164,72.00,1198.00,1493.00,1968.000000,5998.00
Power,7253.0,112.765214,52.844178,34.20,77.00,98.60,138.030000,616.00
Seats,7253.0,5.279722,0.808688,0.00,5.00,5.00,5.000000,10.00
New_Price,7253.0,21.307322,24.256314,3.91,7.88,11.30,21.690000,375.00
Price,7253.0,9.479468,10.191695,0.44,3.85,6.75,9.479468,160.00


<IPython.core.display.Javascript object>

- `Mileage`: min is 0.00 indicating to a new Car, where max mileage is almost 34k
- `Engine`: Normally, a car's cc can range between 800 cc to 6000 cc. As mentioned earlier, the lower the cc of the car, the better the mileage obtained. Hence, cars with up to 1000 cc give the best mileage, while cars with the cc ranging from 1000 to 1800 are also quite economical.Our max cc value doesn't seem overly exagurated therefore missing values treatment does not seem to have introduced an outlier
- `Power`: Generally, anything more than 200bhp can reasonably be considered a high-performance car, in our case we do see some luxury vehicles with max 616 bhp, this will be further analyed in outlier treatment
- `Seats`: While most car brands whether ecnomical or luxury can have 5 seats, however 10 seats is usually for `2022 Ford Transit Passenger Van` type vehicles, there are seems to be some outliers here
- `Price`: Less than 75% of vehicles are resonably priced at less than 10K, however there are few outliers that has 160K, which are probably correl

**Encoding categorical columns** 

 # **Sanity Check**
 _______________________________________________________

`@author mirali - Created on Friday Feb 3 - Start your work from the next line`

# **EDA - Exploratory Data Analysis on PreProcessed Data**
_______________________________________________________
1) Independent Variable must always be Normally distributed in order to generalize effectively and not be biased 
   - If not normally distirbuted we need to perform transformation on data

`@author mirali - Created on Friday Feb 3 - Start your work from the next line`

<h3 style="color:Brown">Bivariate Analysis</h3>

<h3 style="color:Brown">Univariate Analysis</h3>

### <a id='link1'>Summary of EDA</a>

# **Build a linear regression model to predict the prices of used cars**
_______________________________________________________

`@author mirali - Created on Friday Feb 3 - Start your work from the next line`

### <a id='link1'>Conclusions</a>

# **Insights and Recommendations that will help the business**
_______________________________________________________

`@author mirali - Created on Friday Feb 3 - Start your work from the next line`